<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/Theo/Theo_Dacon_satellite_baseline_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os.path

#if not os.path.isfile('train.npy'):
#    !cp '/content/drive/My Drive/2020 Kaggle Study/data/train.npy' train.npy
if not os.path.isfile('test.npy'): 
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/test.npy' test.npy
if not os.path.isfile('gmi_preci.npy'):
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci.npy' gmi_preci.npy

In [3]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.layers import LeakyReLU
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
 
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#train = np.load('/content/train.npy')  # float32
test = np.load('/content/test.npy')  # float64
gmi_preci = np.load('/content/gmi_preci.npy')  # float32

## 1. Data collection and preprocessing

In [0]:
def preprocess(data):
    land_type_data = data[:,:,:,9]
    data[:,:,:,9] = np.where(land_type_data//100 == 2, 0.7,
                             np.where(land_type_data//100 == 3, 0.3,
                                      land_type_data//100))
    return data[:,:,:,9]

In [0]:
# 1.1 Data Collection
# train
train = np.load('/content/drive/My Drive/2020 Kaggle Study/data/train.npy')[:,:,:,:12]

# feature
train_temp = train[:,:,:,:9]

train_landtype = preprocess(train)
train_lon_GMI = train[:,:,:,10]
train_lat_GMI = train[:,:,:,11]
train_DEM = np.load("/content/drive/My Drive/2020 Kaggle Study/data/DEM_DPR/dem_train_DPR.npy")

# target
train_rain = np.load("/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci.npy")

# feature
test_temp = test[:,:,:,:9]

test_landtype = preprocess(test)
test_lon_GMI = test[:,:,:,10]
test_lat_GMI = test[:,:,:,11]
test_DEM = np.load("/content/drive/My Drive/2020 Kaggle Study/data/DEM_DPR/dem_test_DPR.npy")

all_temp = np.concatenate((train_temp, test_temp), axis = 0)
del train_temp, test_temp
all_landtype = np.concatenate((train_landtype, test_landtype), axis = 0)
del train_landtype, test_landtype
all_lon_GMI = np.concatenate((train_lon_GMI, test_lon_GMI), axis = 0)
del train_lon_GMI, test_lon_GMI
all_lat_GMI = np.concatenate((train_lat_GMI, test_lat_GMI), axis = 0)
del train_lat_GMI, test_lat_GMI
all_DEM = np.concatenate((train_DEM, test_DEM), axis = 0)
del train_DEM, test_DEM

In [0]:
all_temp.shape

In [0]:
# 1.2 Data preprocessing
def minmaxscaler(data_info, n):
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  data_in = data_info.reshape(-1, 1)
  scaler.fit(data_in)
  data_in = scaler.transform(data_in)
  data_in = data_in.reshape(-1)
  data_info = data_in.reshape(n, 40, 40, 1)
  return data_info

def standardscaler(data_info):
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  data_in = data_info.reshape(-1, 1)
  scaler.fit(data_in)
  data_in = scaler.transform(data_in)
  data_in = data_in.reshape(-1)
  data_info = data_in.reshape(78761, 40, 40, 1)
  return data_info

In [0]:
# temp - standard , land - categorization, dem - minmax, rain - original
print("temp")
for i in range(0,9):
  t_temp = standardscaler(all_temp[:,:,:,i])
  print("cat : %d" %i)
  all_temp[:,:,:,i:i+1] = t_temp

#print("land")
#t_land = all_landtype.reshape(-1)
#for i in range(len(t_land)):
#  t_land[i] = int(str(t_land[i])[0])
#all_landtype = t_land.reshape(78761,40, 40, 1)

print("DEM")
all_DEM = minmaxscaler(all_DEM, 78761)

print("GMI lon, lat")
all_lon_GMI = standardscaler(all_lon_GMI)
all_lat_GMI = standardscaler(all_lat_GMI)

#print("train rain")
#train_rain = np.where(train_rain>0, train_rain, 0)
#train_rain = minmaxscaler(train_rain, 76345)

In [0]:
all_landtype = all_landtype.reshape(78761, 40, 40, 1)

In [0]:
train = np.concatenate((all_temp[:76345,:,:,:], all_landtype[:76345,:,:,:], all_DEM[:76345,:,:,:], train_rain), axis = 3)
test = np.concatenate((all_temp[76345:,:,:,:], all_landtype[76345:,:,:,:], all_DEM[76345:,:,:,:]), axis = 3)

In [0]:
del all_temp, all_landtype, all_DEM, all_lon_GMI, all_lat_GMI, train_rain

In [0]:
# -9999를 포함한 이미지, 강수인 지역이 50픽셀 미만인 이미지 제거
is_valid = (train[:,:,:,-1].reshape(-1, 1600) < 0).sum(axis=1) == 0
is_valid = is_valid & ((train[:,:,:,-1].reshape(-1, 1600) >= 0.1).sum(axis=1) >= 50)
train = train[is_valid]  # (-1, 40, 40, 15)

## Augmentation

In [0]:
def r(a, n=1):
    return np.rot90(a, n, (1, 2))

def f_ud(a):
    return np.flip(a, 1)

def f_lr(a):
    return np.flip(a, 2)

def t(a):
    return np.transpose(a, (0, 2, 1, 3))

In [0]:
train = tf.convert_to_tensor(train, dtype=np.float32)
train = train.numpy()
test = tf.convert_to_tensor(test, dtype=np.float32)
test = test.numpy()

In [0]:
train = np.concatenate((train, r(f_lr(train)), f_lr(train)))

In [0]:
train = tf.convert_to_tensor(train, dtype=np.float32)
train = train.numpy()

In [19]:
print(train.shape)
print(test.shape)

(103113, 40, 40, 12)
(2416, 40, 40, 11)


## 2. 모델 생성

In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

In [0]:
def build_model(input_layer_c, input_layer_d, start_neurons):

    # 40 x 40 -> 20 x 20
    convc1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(input_layer_c)
    convc1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(convc1)
    poolc1 = BatchNormalization()(convc1)
    poolc1 = MaxPooling2D((2, 2))(poolc1)
    poolc1 = Dropout(0.25)(poolc1)
    # 20 x 20 -> 10 x 10
    convc2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding="same")(poolc1)
    convc2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding="same")(convc2)
    poolc2 = BatchNormalization()(convc2)
    poolc2 = MaxPooling2D((2, 2))(poolc2)
    poolc2 = Dropout(0.25)(poolc2)
    convmc = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding="same")(poolc2)

    # 40 x 40 -> 20 x 20
    convd1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(input_layer_d)
    convd1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(convd1)
    poold1 = BatchNormalization()(convd1)
    poold1 = MaxPooling2D((2, 2))(poold1)
    poold1 = Dropout(0.25)(poold1)
    # 20 x 20 -> 10 x 10
    convd2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding="same")(poold1)
    convd2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding="same")(convd2)
    poold2 = BatchNormalization()(convd2)
    poold2 = MaxPooling2D((2, 2))(poold2)
    poold2 = Dropout(0.25)(poold2)
    convmd = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding="same")(poold2)

    convm = concatenate([convmc, convmd], axis = 3) 
    
    # 5 x 5 -> 10 x 10
#    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), activation='relu', padding="same")(convm)
#    uconv3 = concatenate([deconv3, convc3, convd3], axis = 3)
#    uconv3 = Dropout(0.25)(uconv3)
#    bn = BatchNormalization()(uconv3)
#    uconv3 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(bn)
#    uconv3 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(uconv3)
#    uconv3 = BatchNormalization()(uconv3)

    # 5 x 5 -> 10 x 10
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), activation='relu', padding="same")(convm)
    uconv2 = concatenate([deconv2, convc2], axis = 3)
    uconv2 = Dropout(0.25)(uconv2)
    uconv2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same", activation = 'relu')(uconv2)
    uconv1 = concatenate([deconv1, convc1], axis = 3)
    uconv1 = Dropout(0.25)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1),activation='relu', padding="same")(uconv1)
    
    return output_layer

In [0]:
input_layer_c = Input((40, 40, 9))
input_layer_d = Input((40, 40, 2))
output_layer = build_model(input_layer_c, input_layer_d,  32)

In [0]:
model = Model(inputs = [input_layer_c, input_layer_d], outputs = [output_layer])

In [0]:
from sklearn.metrics import f1_score
def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))
def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))
def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)
def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score
def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="mae", optimizer=opt, metrics=[maeOverFscore_keras, fscore_keras])

In [28]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
model_history = []
fold_number = 0

for train_index, test_index in kf.split(train):
    print(f'\nFold {fold_number}')
    fold_number = fold_number + 1
    
#-----------------------------------------------------------------------------
    def trainGenerator():
        print(1)
        for file in train[train_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:11]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels

    train_dataset = tf.data.Dataset.from_generator(trainGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 2])), tf.TensorShape([40,40,1])))
    
    train_dataset = train_dataset.batch(128).prefetch(1)
#-----------------------------------------------------------------------------
    def testGenerator():
        print(2)
        for file in train[test_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:11]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels
    test_dataset = tf.data.Dataset.from_generator(testGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 2])),tf.TensorShape([40,40,1])))
    test_size = len(test_index)
    test_dataset = test_dataset.batch(test_size).prefetch(1)
#-----------------------------------------------------------------------------
    history = model.fit(train_dataset, epochs = 5, verbose=1, validation_data = test_dataset)
    model.save("/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5")
    #history = model.fit(train_dataset, epochs = 5, ,batch_size = 11, verbose=1)
    model_history.append(history)
    #del data
    del train_dataset
    del test_dataset
    gc.collect()


Fold 0
Epoch 1/5
1
    645/Unknown - 320s 496ms/step - loss: 0.2294 - maeOverFscore_keras: 2.0508 - fscore_keras: 0.66512
645/645 [==============================] - 379s 588ms/step - loss: 0.2294 - maeOverFscore_keras: 2.0508 - fscore_keras: 0.6651 - val_loss: 0.2125 - val_maeOverFscore_keras: 1.9361 - val_fscore_keras: 0.6676
Epoch 2/5
1
645/645 [==============================] - 388s 602ms/step - loss: 0.2162 - maeOverFscore_keras: 1.8415 - fscore_keras: 0.6917 - val_loss: 0.2118 - val_maeOverFscore_keras: 1.8621 - val_fscore_keras: 0.6884
Epoch 3/5
1
645/645 [==============================] - 392s 607ms/step - loss: 0.2082 - maeOverFscore_keras: 1.6932 - fscore_keras: 0.7200 - val_loss: 0.2053 - val_maeOverFscore_keras: 1.7767 - val_fscore_keras: 0.6996
Epoch 4/5
1
645/645 [==============================] - 398s 617ms/step - loss: 0.2033 - maeOverFscore_keras: 1.6154 - fscore_keras: 0.7348 - val_loss: 0.1960 - val_maeOverFscore_keras: 1.6018 - val_fscore_keras: 0.7374
Epoch 5/5
1
6

In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5', custom_objects={"maeOverFscore_keras": maeOverFscore_keras,"fscore":fscore, "fscore_keras":fscore_keras })

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss="mae", optimizer=opt, metrics=[maeOverFscore_keras, fscore_keras])

In [0]:
train_2 = np.concatenate((r(train), r(train, 3), f_ud(train)))

In [0]:
train_2 = tf.convert_to_tensor(train_2, dtype=np.float32)
train_2 = train_2.numpy()

In [28]:
train_2.shape

(103113, 40, 40, 12)

In [29]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
model_history = []
fold_number = 0

for train_index, test_index in kf.split(train_2):
    print(f'\nFold {fold_number}')
    fold_number = fold_number + 1
    
#-----------------------------------------------------------------------------
    def trainGenerator():
        print(1)
        for file in train_2[train_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:11]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels

    train_dataset = tf.data.Dataset.from_generator(trainGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 2])), tf.TensorShape([40,40,1])))
    
    train_dataset = train_dataset.batch(128).prefetch(1)
#-----------------------------------------------------------------------------
    def testGenerator():
        print(2)
        for file in train_2[test_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:11]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels
    test_dataset = tf.data.Dataset.from_generator(testGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 2])),tf.TensorShape([40,40,1])))
    test_size = len(test_index)
    test_dataset = test_dataset.batch(test_size).prefetch(1)
#-----------------------------------------------------------------------------
    history = model.fit(train_dataset, epochs = 5, verbose=1, validation_data = test_dataset)
    model.save("/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5")
    #history = model.fit(train_dataset, epochs = 5, ,batch_size = 11, verbose=1)
    model_history.append(history)
    #del data
    del train_dataset
    del test_dataset
    gc.collect()


Fold 0
Epoch 1/5
1
    645/Unknown - 311s 482ms/step - loss: 0.1850 - maeOverFscore_keras: 1.3892 - fscore_keras: 0.77552
645/645 [==============================] - 365s 566ms/step - loss: 0.1850 - maeOverFscore_keras: 1.3892 - fscore_keras: 0.7755 - val_loss: 0.1884 - val_maeOverFscore_keras: 1.4411 - val_fscore_keras: 0.7735
Epoch 2/5
1
645/645 [==============================] - 388s 602ms/step - loss: 0.1842 - maeOverFscore_keras: 1.3810 - fscore_keras: 0.7766 - val_loss: 0.1876 - val_maeOverFscore_keras: 1.4306 - val_fscore_keras: 0.7754
Epoch 3/5
1
645/645 [==============================] - 393s 610ms/step - loss: 0.1838 - maeOverFscore_keras: 1.3784 - fscore_keras: 0.7767 - val_loss: 0.1877 - val_maeOverFscore_keras: 1.4353 - val_fscore_keras: 0.7739
Epoch 4/5
1
645/645 [==============================] - 423s 657ms/step - loss: 0.1832 - maeOverFscore_keras: 1.3721 - fscore_keras: 0.7776 - val_loss: 0.1865 - val_maeOverFscore_keras: 1.4130 - val_fscore_keras: 0.7773
Epoch 5/5
1
6

In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5', custom_objects={"maeOverFscore_keras": maeOverFscore_keras,"fscore":fscore, "fscore_keras":fscore_keras })

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="mae", optimizer=opt, metrics=[maeOverFscore_keras, fscore_keras])

In [0]:
train_3 = np.concatenate((t(r(train, 2)), r(train, 2), train))

In [0]:
train_3 = tf.convert_to_tensor(train_3, dtype=np.float32)
train_3 = train_3.numpy()

In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
model_history = []
fold_number = 0

for train_index, test_index in kf.split(train_3):
    print(f'\nFold {fold_number}')
    fold_number = fold_number + 1
    
#-----------------------------------------------------------------------------
    def trainGenerator():
        print(1)
        for file in train_3[train_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:11]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels

    train_dataset = tf.data.Dataset.from_generator(trainGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 2])), tf.TensorShape([40,40,1])))
    
    train_dataset = train_dataset.batch(128).prefetch(1)
#-----------------------------------------------------------------------------
    def testGenerator():
        print(2)
        for file in train_3[test_index]:
            target= file[:,:,-1:].reshape(40,40,1)
            cutoff_labels = np.where(target < 0, 0, target)
            feature_c = file[:,:,:9]
            feature_d = file[:,:,9:11]
            if (cutoff_labels > 0).sum() < 50:
                continue
            yield (feature_c, feature_d), cutoff_labels
    test_dataset = tf.data.Dataset.from_generator(testGenerator,
                                                ((tf.float32, tf.float32), tf.float32),
                                                ((tf.TensorShape([40,40,9]), tf.TensorShape([40, 40, 2])),tf.TensorShape([40,40,1])))
    test_size = len(test_index)
    test_dataset = test_dataset.batch(test_size).prefetch(1)
#-----------------------------------------------------------------------------
    history = model.fit(train_dataset, epochs = 5, verbose=1, validation_data = test_dataset)
    model.save("/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5")
    #history = model.fit(train_dataset, epochs = 5, ,batch_size = 11, verbose=1)
    model_history.append(history)
    #del data
    del train_dataset
    del test_dataset
    gc.collect()


Fold 0
Epoch 1/5
1
    645/Unknown - 342s 531ms/step - loss: 0.1809 - maeOverFscore_keras: 1.3472 - fscore_keras: 0.78172
645/645 [==============================] - 405s 628ms/step - loss: 0.1809 - maeOverFscore_keras: 1.3472 - fscore_keras: 0.7817 - val_loss: 0.1823 - val_maeOverFscore_keras: 1.3712 - val_fscore_keras: 0.7848
Epoch 2/5
1
645/645 [==============================] - 380s 589ms/step - loss: 0.1803 - maeOverFscore_keras: 1.3410 - fscore_keras: 0.7828 - val_loss: 0.1825 - val_maeOverFscore_keras: 1.3627 - val_fscore_keras: 0.7858
Epoch 3/5
1
645/645 [==============================] - 387s 599ms/step - loss: 0.1799 - maeOverFscore_keras: 1.3370 - fscore_keras: 0.7835 - val_loss: 0.1823 - val_maeOverFscore_keras: 1.3653 - val_fscore_keras: 0.7850
Epoch 4/5
1
645/645 [==============================] - 394s 610ms/step - loss: 0.1796 - maeOverFscore_keras: 1.3340 - fscore_keras: 0.7841 - val_loss: 0.1820 - val_maeOverFscore_keras: 1.3598 - val_fscore_keras: 0.7864
Epoch 5/5
1
6

In [0]:
test_o = np.load('/content/drive/My Drive/2020 Kaggle Study/data/test.npy') 

In [20]:
test_o.shape

(2416, 40, 40, 14)

In [0]:
dem_test = test[:,:,:,10:11]

In [0]:
test_ = np.concatenate((test[:,:,:,:9], test[:,:,:,9:10], test_o[:,:,:,10:]), axis = 3)

In [23]:
test_.shape

(2416, 40, 40, 14)

In [0]:
dr = [(-1, -1), (-1, 0), (-1, 1),
      (0, -1), (0, 0), (0, 1),
      (1, -1), (1, 0), (1, 1)]

def get_dist(p1, p2):  # p1, p2: shape=(-1, 2).
    x1, y1 = np.deg2rad(p1[:,0]), np.deg2rad(p1[:,1])
    x2, y2 = np.deg2rad(p2[:,0]), np.deg2rad(p2[:,1])
    dlon = x2 - x1
    dlat = y2 - y1
    a = np.sin(dlat/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dlon/2)**2 
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))    
    return 6373.0 * c  # km, shape=(-1).

# value: (40, 40, -1)
# ori_ll, tgt_ll: (40, 40, 2)
def compen_ll(value, ori_ll, tgt_ll):  # ori_ll에서의 value를 tgt_ll에 대한 값으로 바꿈
    ret = np.empty_like(value)
    n, m = value.shape[0], value.shape[1]
    for i in range(n):
        for j in range(m):
            nears = []  # (row, col, value)
            for k in range(9):
                ii = i + dr[k][0]
                jj = j + dr[k][1]
                if ii >= 0 and ii < n and jj >= 0 and jj < m:
                    nears.append((ori_ll[ii, jj][0], ori_ll[ii, jj][1],
                                  tgt_ll[i, j][0], tgt_ll[i, j][1],
                                  value[ii, jj]))
            nears = np.array(nears)  # shape=(-1, 5)
            dists = get_dist(nears[:, 0:2], nears[:, 2:4]).reshape(-1, 1)
            values = nears[:, 4].reshape(-1, 1)
            nears = np.concatenate((dists, values), 1)
            nears = nears[np.argsort(nears[:, 0])]  # sort by dist
            nears = nears[:2, :]  # 가까운 점 2개만 고려
            
            weights = 1 / (nears[:, 0] ** 2 + sys.float_info.epsilon)
            weighted_sum = (weights * nears[:, 1]).sum()
            ret[i, j] = weighted_sum / weights.sum()
    return ret

In [0]:
from multiprocessing import Process, Manager

def proc_func(splitted, dpr_preci, proc_id):
    part = splitted[proc_id]
    arr = np.empty_like(part[:, :, :, 14])  # shape=(-1, 40, 40)
    for i in range(part.shape[0]):
        arr[i, :, :] = compen_ll(part[i, :, :, 14], part[i, :, :, 10:12], part[i, :, :, 12:14])
    dpr_preci[proc_id] = arr

def gmi2dpr(test, gmi_preci):
    n_procs = 4
    procs = []
    manager = Manager()
    dpr_preci = manager.list([None] * n_procs)

    data = np.concatenate((test, gmi_preci), axis=3)
    n_imgs = data.shape[0]  # split data into n_procs arrays
    splitted = np.split(data, np.arange(n_imgs // n_procs + n_imgs % n_procs, n_imgs, n_imgs // n_procs))

    for proc_id in range(n_procs):
        proc = Process(target=proc_func, args=(splitted, dpr_preci, proc_id))
        proc.start()
        procs.append(proc)

    for proc in procs:
        proc.join()

    dpr_preci = np.concatenate(dpr_preci)
    return dpr_preci

In [0]:
x_test = test[:, :, :, :10]
test_ = np.concatenate((test[:,:,:,:10], test_o[:,:,:,10:]), axis = 3)

In [0]:
import pandas as pd
import numpy as np
import random
import sys
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, BatchNormalization, concatenate, Input
from tensorflow.keras import Model

In [0]:
pred=model.predict([test[:,:,:,:9],test[:,:,:,9:11]])

In [91]:
pred.shape

(2416, 40, 40, 1)

In [0]:
pred = gmi2dpr(test_, pred)

In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/2020 Kaggle Study/Theos/Model/three_leg_CNN_5.h5', custom_objects={"fscore_keras": fscore_keras,"maeOverFscore_keras": maeOverFscore_keras,"fscore":fscore })

In [0]:
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')

In [0]:
pred = np.where(pred==0, 0.099, pred)

In [0]:
submission.iloc[:,1:] = pred.reshape(-1, 1600)

In [0]:
submission.to_csv('/content/drive/My Drive/2020 Kaggle Study/Theos/Submission/three_leg_CNN_final2.csv', index = False)